# Curso de problemas prácticos de Machine Learning

In [1]:
import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import nltk # natural lenguage toolt kit, procesamiento de lenguaje natural. Tiene informaicón extra para trabajar con esa info.
from nltk.corpus import stopwords
from unidecode import unidecode

In [2]:
# Descarga información necesaria para el pre-procesamiento
nltk.download('stopwords') # Paquete de datos extra y queda disponible 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# 1.- Donde diga AMLO dice político, de lo contrario dice médico

In [4]:
dialogs = pd.read_csv("dialogos.csv", index_col=0)
dialogs["speaker"] = np.where(dialogs["speaker"]== "amlo","politico", "medico") # 1
dialogs.head()


,speaker,dialog,length
0,politico,"Amigas, amigos, paisanas, paisanos de Palenque:",47
1,politico,Me da mucho gusto estar de nuevo en trabajos d...,91
2,politico,"Antes de entrar en materia, quiero enviar un s...",471
3,politico,Di la instrucción de que se mantengan trabajan...,338
4,politico,"Quiero decirles a mis paisanos que padecen, qu...",113


In [5]:
# ¿Qué problema estamos atacando con ML?
# Una clasificación porqué 'o es uno o es el otro' y es binaria porque son dos posibles respuestas.
# ¿Es medico o político?

In [6]:
# Una vez encontrado el problema hay que encontrar la métrica para evaluar el problema.
# Usualmente hay 2 tipos de metrica:
# 1.- Las que se refieren al negocio. P/E: ¿Cuantos dialogos podemos detectar que pertenecen a un político?
# 2.- Las que se refieren directamente al ML

### Métrica de elección: Acurrency

In [7]:
# Antes de cualquier modelo, debemos hacer analisis exploratorio de datos.

### Dividir data set

In [8]:
# Antes de continuar con el ML es super importante dividir el Dataset

# Queremos guardar una parte del data para entrenar el modelo.
# Otra parte se le conoce como 'conjunto de validación' el cual nos permite ver el desemepeño de nuetro modelo para tomar decisiones.
# Y guardamos otra parte más grande para entrenar el modelo.

In [9]:
# 1.- Usamos una fucnión desklearn que esta dentro de sklearn.model_selection
# 2.- Lo primero que voy a hacer es dividir mi conjunto dataset dialogs
# quiero utilizar un 20% para prueba para probar el acurrancy de mi modelo
# Quiero que la division sea proporcional y que utilice las proporciones de la columna speaker para quela division sea proporcional
# 3.- Regresa dos conjuntos de datos
# 4.- Después voy a volver a dividir el conjunto rest entre el conjunto de entrenamiento y el de validaacion con un 20% de esos datos
# 5.- Tengo casi 100 mil valores para entrenar modelo, casi 25 mil para validar y 31 mil para hacer pruebas

In [10]:
from sklearn.model_selection import train_test_split #1

In [11]:
rest, test = train_test_split(dialogs, test_size=0.2, stratify=dialogs["speaker"]) #2
# 3, se ponen dos variables

In [12]:
train, val = train_test_split(rest, test_size=0.2, stratify=rest["speaker"]) # 4

In [13]:
len(train), len(val), len(test)  # 5

(99524, 24881, 31102)

In [14]:
# Guardo en varibles todos los valores

In [15]:
dialogs_train = train["dialog"]
dialogs_val = val["dialog"]
dialogs_test = test["dialog"]

target_train = train["speaker"]
target_val = val["speaker"]
target_test = test["speaker"]

In [16]:
target_train

37836       medico
59609       medico
109596    politico
84745     politico
82818     politico
            ...   
145670    politico
17532       medico
12604     politico
86729     politico
98161     politico
Name: speaker, Length: 99524, dtype: object

## Feature Engineering

### Etiquetas

In [17]:
# Debemos hacer que el modelo pueda entender, y eso es con números

In [18]:
# Primero vamos a convertir nuestra etiqueta en números
# 'WHERE': En donde la variable sea igual a politico convetrias a 1 y sino será 0

In [19]:
train_y = np.where(target_train == "politico", 1,0)
val_y = np.where(target_val == "politico", 1,0)
test_y = np.where(target_test == "politico", 1,0)

### Texto

In [20]:
# Convertir texto a números
# 1.- Damos un vistazo al data
# 3.- Tomamos una frase como ejemplo

In [21]:
dialogs_train.sample(10, random_state=132).values #1

array(['En los últimos dos sexenios hubo condonaciones a un grupo muy reducido de 400 mil millones de pesos. Entonces, eso ya se terminó.',
       'Apenas se expresó la democracia en periodos, fueron momentos estelares, pero por lo general lo que imperó por siglos fue la imposición, son tres siglos de dominación colonial, ni modo que hubiera democracia.',
       'En general, si está por debajo de 93 por ciento habla de incapacidad para oxigenar apropiadamente el cuerpo. Esto es particularmente importante porque, si empieza a bajar, si en un mismo día se recude dos puntos porcentuales, puede ser una señal importante de alerta que requiera que acudas al hospital para ser atendido.',
       'Son dos enfermedades diferentes, causadas por distintos virus. El COVID es causado por el virus SARS-CoV-2, un virus nuevo que entró a la especie humana, por lo menos desde que se tiene registro a finales de 2019, en la última semana, y que no había estado presente entre los humanos.',
       'Entonce

In [22]:
example_sentence = dialogs_train.iloc[80567]
print(example_sentence) #3

De lo contrario se tendría que mandarse la licitación, pero antes de la licitación se tendría que pasar por dos, tres instancias y es un proceso que lleva un mes. Eso no, están llevando los trámites, cuando mucho, un día. Por eso, les digo estamos trabajando de tiempo completo.


### Tokenizacion

In [23]:
# Lo mas comun es que un token sea una palabra pero no siempre es así, puede ser un emoji, un signo de interrogación, etc.
# 1.- Cada palabra será un ejemplo de toke. cada uno con sus respectivas caracteristicas es 'unico'
# depende elcontexto y los datos que tan estilizados los usaremos, es decir, quita comas, acentos, etc.
# 2.- Utiliza un tokenizador para ayduarse, 'ToktokTokenizer' es mas común con el español, tambien existen algunos para redes scoailes.
# 3.- Utiliza el '#' para mostrar la separación de cada palabra, incluye comas.
# Notas.- Las 'Stopwords' o las palabras comunes son aquellas que no importa quien sea siempre s eusan, ejemplo: el, la los, o, y, etc.

# El idioma tmabien define como hacer la tokenización


In [24]:
# example_sentence.split()

In [25]:
from nltk.tokenize.toktok import ToktokTokenizer #2

tk_tokenizer = ToktokTokenizer() # Ojeto que es ToktokTokenizer

In [26]:
tokens = tk_tokenizer.tokenize(example_sentence + "?")
print(" # ".join(tokens)) # 3

De # lo # contrario # se # tendría # que # mandarse # la # licitación # , # pero # antes # de # la # licitación # se # tendría # que # pasar # por # dos # , # tres # instancias # y # es # un # proceso # que # lleva # un # mes. # Eso # no # , # están # llevando # los # trámites # , # cuando # mucho # , # un # día. # Por # eso # , # les # digo # estamos # trabajando # de # tiempo # completo. # ?


In [27]:
# Vamos a tratar de limpair el data, y hacer que el algoritmo generalice mas al limpiar la cadena.

# Primero vamos a conseguir un monton de stopwords
# 1.- Traemo todos los stopwords del idioma español (esta es una cadena)
# 2.- Conseguir todos los modulos de puntuación con el modulo string (esta es una lista)
# 2.- Uso los '¿¡' porque trabajo con el español y son los signos de apertura.
# 3.- Voy a combinar las dos en uno solo set de tokenes no deseados
# 4.- Creo un tokenizer (objeto)
# Notas.- 'unidecode' permite eliminar acentos, funciona con más idiomas que el español, con '¨'


In [33]:
from nltk.corpus import stopwords
sp_stopwords = stopwords.words("spanish") # 1
sp_punctuation = string.punctuation + '¿¡'    # 2

no_deseados = set((unidecode(word) for word in sp_stopwords)) | set(sp_punctuation) # 3

tk_tokenizer = ToktokTokenizer() # 4


In [34]:
sp_stopwords

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [35]:
no_deseados  # Lo convierto en set porque es más eficiente que una lista, mejora en la función.

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eramos',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estabamos',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estais',
 'estamos',
 'estan',
 'estando',
 'estar',
 'estara',
 'estaran',
 'estaras',
 'estare',
 'estareis',
 'estaremos',
 'estaria',
 'estariais',
 'estariamos',
 'estarian',
 'estarias',
 'estas',
 'este',
 'esteis',
 'estemos',
 'esten',
 'estes',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieramos',
 'estuvieran',
 'estuvieras',
 'estu

In [31]:
# 1.- Creo una funcion que reciba una oración
# 2.- creo una lista vacía en donde guardare los tokens que sí queiro conservar de mi conjunto de esta oración
# 3.- Agarro mi oración y con unidecoe elimino acentos o caracteres que no quiero y la guardo el clean_sentence
# 4.- Tokenizar esa oración que esta limpia y nos va a regresar cada oración separada (reecordar lo del '#')
# 5.- Quiero convertir esos token en minisucula porque sino reconocera plaabras con mayusculua o miniscula como diferente aunque sean las mismas
# 6.- Uso mi conjunto de datos de toknes no deseados  y uno, si mi token esta entre los no deseados voy a continuar
# 7.- Sino de otro modo voy a añadir este token
# 8.- Y al final voya  regresar limpia
# 9.- Ya tengo mi dataset limpio, con las palabras que me interesan


In [36]:
# Función de tokenizador:
def tokenise(sentence):   # 1
    limpia = []  # 2
    clean_sentence = unidecode(sentence) # 3
    for token_ in tk_tokenizer.tokenize(clean_sentence): # 4
        token = token_.lower() # 5
        if token in no_deseados: # 6
            continue
        limpia.append(token)  # 7
    return limpia # 8

In [37]:
print(tokenise(example_sentence))  # 9

['contrario', 'mandarse', 'licitacion', 'licitacion', 'pasar', 'dos', 'tres', 'instancias', 'proceso', 'lleva', 'mes.', 'llevando', 'tramites', 'dia.', 'digo', 'trabajando', 'tiempo', 'completo']


In [ ]:
# Seguimos con texto, nuestro algoritmo aún no puede reconocerlo.

### Vectorización

In [ ]:
# Conversión a números
# Todo debe quedar grabado, es decir, de manera clara cual fue el proceso de tokenización.
# Muchos métodos, iniciamos con el más fácil.
# Tendremos una tabla enorme, donde cada columna será cada uno de los token, una para cada palabra: 'contrario', 'mandarse', 'licitacion', etc.
# Cada fila va a ser cada una de las oraciones que tenemos



- One Hot Encoding

In [ ]:
# Ejemplo: 
# 1. viva mexico paisanos septiembre
# 2. mexico inundaciones viva voz
# Obtendríamos algo como esto:

#     viva	mexico	paisanos	...	septiembre	inundaciones	voz
# 1	   1	   1	   1	    ...    	1       	0	         0
# 2	   1	   1	   0	    ...	    0	        1            1

In [41]:
# One Hot encoding
# Significa que ponemos un 1 donde hay una ocurrencia
# No es solo para texto, se puede usar para convertir variables categoricas en números.
# Es valor booleano, existe o no, no es cantidad de veces.



In [ ]:
# 1.- De sklearn de su modulo feature_extraccion tiene un submodulo dedicado a texto, voy autilizar algo que se llama CountVectorizer
# 2.- Estoy creando un objeto de tipo binario, que solo deje 1 y 0
# Nota.- True es para que sea binario, analyzer pasamos funcion que utilice para tokenizar nuestras palabras
# Nota.- max_feautre le estoy diciendo cuantas columnas quiero que tenga mi tabla final
# Nota.- Si pusiera 'False' contaría ocurrencias
# 3.-

# Sí el modelo tarda mucho en entrenar puedo poner un límite a mi tabla para los tokens mas frecuentes.


In [ ]:
# vectorizador_ejemplo = CountVectorizer(binary=True, analyzer=tokenise, max_features=4000) 

In [42]:
from sklearn.feature_extraction.text import CountVectorizer  # 1

vectorizador_ejemplo = CountVectorizer(binary=True, analyzer=tokenise)  # 2

In [43]:
# Ejemplo!!
ejemplos = [
    "viva mexico paisanos en setpiembre",
    "en mexico hay inundaciones de viva voz"
]
vectors = vectorizador_ejemplo.fit_transform(ejemplos)

vocabulary = vectorizador_ejemplo.vocabulary_
columns = [token for token, _ in sorted(vocabulary.items(), key=lambda item: item[1])]
pd.DataFrame(vectors.todense(), columns=columns, index=[1, 2])

,inundaciones,mexico,paisanos,setpiembre,viva,voz
1,0,1,1,1,1,0
2,1,1,0,0,1,1


In [44]:
vectorizador_real = CountVectorizer(binary=True, analyzer=tokenise, max_features=1000)

In [ ]:
# 1.- Lo primero que debemos hacer es preparar con el método fit, es como llenar la tabla con 0 y 1
# 2.- Ahora uso transform para tranformar mis datos de entrenamiento, validacion y conjunto de pruebas


In [45]:
vectorizador_real.fit(dialogs_train)  # 1

CountVectorizer(analyzer=<function tokenise at 0x000001EC659BBA60>, binary=True,
                max_features=1000)

In [46]:
# 2
train_x = vectorizador_real.transform(dialogs_train)
val_x = vectorizador_real.transform(dialogs_val)
test_x = vectorizador_real.transform(dialogs_test)

In [ ]:
# 1.- Regresa una matriz dispersa
# Muchos tokens, pero las oraciones no tienen tokens, por eso lo guarda en matriz


In [47]:
train_x  # 1

<99524x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 932922 stored elements in Compressed Sparse Row format>

### Modelado

In [ ]:
# Aquí ya se escoge el modelo

# 1.- Se utilizará una regresión logistica (se usa muhco, es simple) facil de interpretar
# 2.- Creo un objeto de tipo LogisticRegresion
# 3.- Decirle .fit quiero que entrenes este modelo utilizando este conjunto de datos de entrada (ya convertido en números), y quiero que utilices estas
# etiquetas para que realices esa evaluación.
#

In [48]:
from sklearn.linear_model import LogisticRegression  # 1

In [49]:
lr = LogisticRegression()  # 2

In [50]:
# Aquí comienza el ML
lr.fit(train_x, train_y)  # 3

c:\Users\User\Desktop\ciencia-datos\env\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Vamos a revisar el desempeño que no tenga overfeating y que cambios podríamos hacerle al modelo
# Ya lo enrené y ahora puedo usar 'predict' para que me diga cuales son los valores que predijo a partir
# de mi conjunto de datos de entrenamiento y mi conjunto de datos de validación

# 1.- Aqui ya da 1 y 0 porque una vez que ya entrenó, espero que me de resultados 1 y 0, depende de los valores de entrada


In [51]:
train_pred = lr.predict(train_x)  # Para diagnosticar overfitting
val_pred = lr.predict(val_x)  # Para decidir cambios sobre el modelo

In [52]:
val_pred # 1

array([1, 1, 1, ..., 1, 1, 1])

- ¿Cuál es la métrica que ibamos a usar? : Acurrency

In [57]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Le pasamos los valores verdaderos y los valores predecidos
# Esto solo es el conjunto de datos de entrenamiento
# 1.- El acurrency por debajo del 70% quiere decir que no esta aprendiendo bien
# 2.- Para el overfitting nos fijimaos en el conjunto de validación
# Como los valores de prueba y validación son similares es buena señal.
# Sí el acurrency fuera mucho más grande que el ed validación sería una señal de overfetting


In [54]:
print(accuracy_score(train_y, train_pred))  # 1

0.936879546642016


In [56]:
print(accuracy_score(val_pred, val_y))  # 2

0.9357742855994534


In [ ]:
# El modelo se desemepeña de manera adecuada
# El conjunto de prueba no lo hemos usado, hasta estar conforme con el modelo

# El entrenamiento no es el desempeño real de la producción (pruebas)
# Una vez comodo con nuestro modelo, podemos usar la prueba


- Este valor es el importante

In [59]:
test_pred = lr.predict(test_x)
test_accuracy = accuracy_score(test_y, test_pred)

print(f"Test accuracy: {test_accuracy:0.2%}")

Test accuracy: 93.50%


In [ ]:
# Con el valor anterior ya podemos decir que probé el modelo y lo podemos llevar a producción.
# El valor que se comunica es el del conjunto de prueba!!

### Test Dataset

- Como usar este modelo:

In [ ]:
# 1.- Partimos de una oración
# 2.- Pasamos de textos a 1 y 0
# 3.- Llamamos a nuestra predicción
# Nota.- 0 es doctor, 1 político

In [71]:
oración = "Que pasara con la pandemia de covid 19 el proximo año de elecciones"  # 1
own_x = vectorizador_real.transform([oración])  # 2
result = lr.predict(own_x)  # 3
result

array([1])

In [ ]:
# La frase fue dicha por un político

In [ ]:
# Existe otro metodo que no es predict, disponible en regresion logistica, nos da una probabilidad de pertenencia a cad una de las clases.
# Regresa un estimado de quien dijo la frase

In [72]:
medico, politico = lr.predict_proba(own_x).squeeze()*100
medico, politico

(3.246288180236956, 96.75371181976304)

In [ ]:
# Existe un 96% de probabilidad que la frase se haya dicho por un político